In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# -------------------
# 1. 데이터 불러오기
# -------------------
file_path = "../../data/CRI-BIG5CRIME/CRI-BIG5CRIME.csv"
df = pd.read_csv(file_path)

# -------------------
# 2. 자치구별 정제
# -------------------
# 실제 자치구명만 남기기 (합계, 소계 제외)
df = df[~df["자치구"].isin(["합계", "소계", "전체"])].copy()

# -------------------
# 3. 발생 건수 & 검거율 계산
# -------------------
df["big5crime_occurrence"] = pd.to_numeric(df["합계 발생"], errors="coerce")
df["big5crime_arrest_rate"] = (pd.to_numeric(df["합계 검거"], errors="coerce") /
                               df["big5crime_occurrence"]) * 100

# -------------------
# 4. 점수화 (MinMax Scaling)
# -------------------
scaler = MinMaxScaler(feature_range=(0, 100))

# 발생 건수 점수 (많을수록 위험 ↑)
df["big5crime_occurrence_score"] = scaler.fit_transform(df[["big5crime_occurrence"]])

# 검거율 점수 (높을수록 안전 ↑)
df["big5crime_arrest_score"] = scaler.fit_transform(df[["big5crime_arrest_rate"]])

# -------------------
# 5. 최종 결과 정리
# -------------------
result = df[["자치구", 
             "big5crime_occurrence", "big5crime_occurrence_score",
             "big5crime_arrest_rate", "big5crime_arrest_score"]]

result = result.rename(columns={"자치구": "구이름"})

print(result.head())




# CSV 저장
result.to_csv("../../result/CRI/CRI-BIG5CRIME_result.csv", index=False, encoding="utf-8-sig")


   구이름  big5crime_occurrence  big5crime_occurrence_score  big5crime_arrest_rate  big5crime_arrest_score
1  종로구                  2981                   21.891780             116.907078              100.000000
2   중구                  3348                   29.471293              73.626045               13.743847
3  용산구                  3021                   22.717885              75.074479               16.630476
4  성동구                  2023                    2.106568              69.747899                6.014966
5  광진구                  3424                   31.040892              68.136682                2.803920
